In [1]:
import random
from deap import base, creator, tools, algorithms
#from scoop import futures
#import multiprocessing
import numpy as np 
from itertools import cycle, islice 
from openmc_toy import *
import pandas as pd
from collections import defaultdict 

In [2]:
# minimum fuel amount 
creator.create("FuelMin", base.Fitness, weights=(-1.0,))
creator.create("Ind", list, fitness=creator.FuelMin, keff=1.0)

ind = creator.Ind([0.005,1,1,1,1]) # PF, p1, p2, p3, p4

In [3]:
# openmc input + output
def evalOpenMC(ind):
    # return keff, total PF  
    keff, total_pf = run_openmc(name=str(ind.gen)+'_'+str(ind.num), 
                                total_pf=ind[0],
                                poly_coeff=[ind[1],ind[2],ind[3],ind[4]])
    return (keff, total_pf) #(random.randint(-2,3),ind[0])

In [4]:
def triso_distribution(total_pf, dz, total_core_vol, poly_coeff): 
    """ This function returns discrete pf values 
    """
    T_r5 = 4235e-5
    vol_triso = 4/3*np.pi*T_r5**3
    total_trisos = round(total_pf*total_core_vol/vol_triso)
    z_vals = np.arange(1,dz+1)
    z = poly_coeff[0]*z_vals**3 + poly_coeff[1]*z_vals**2 + poly_coeff[2]*z_vals + poly_coeff[3]
    z_trisos = z/(sum(z))*total_trisos
    pf_z = z_trisos*vol_triso/(total_core_vol/dz)
    return np.array(pf_z)

In [5]:
def f_cycle():
    """ This function returns an individual
    We needed this function to only return polynomials that are above 0 for dz vals 
    """
    dz = 10
    polyval = np.array([-1] * dz)
    dz_vals = np.arange(1,dz+1)
    pf_z = np.array([0.3] * dz)
    while len(polyval[polyval<0]) != 0 and len(pf_z[pf_z > 0.25]): 
        poly = [toolbox.poly(),toolbox.poly(),toolbox.poly(),toolbox.poly()]
        polyval = poly[0]*dz_vals**3 + poly[1]*dz_vals**2 + poly[2]*dz_vals + poly[3]
        total_pf = toolbox.pf()
        pf_z = triso_distribution(total_pf, 10, 1, poly)
    return creator.Ind([total_pf] + poly)

In [6]:
toolbox = base.Toolbox()
toolbox.register('pf',random.uniform,0.005,0.1)
toolbox.register('poly',random.uniform,-1,1)
#func_cycle = [toolbox.pf,toolbox.poly,toolbox.poly,toolbox.poly,toolbox.poly]
#toolbox.register('individual',tools.initCycle,creator.Ind,func_cycle)
#toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('individual',f_cycle)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalOpenMC)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.5, 
                 low=[0.005,-1,-1,-1,-1], up=[0.1,1,1,1,1], indpb=0.5)
toolbox.register("select", tools.selBest, k=10, fit_attr='fitness')
#toolbox.register("select", tools.selTournament, tournsize=3)
# multiproccessing is for single multicore machine 
#pool = multiprocessing.Pool()
#toolbox.register("map", pool.map)
# scoop is for cluster 
#toolbox.register("map", futures.map)

In [7]:
# create pandas dataframe for results summary 
results = defaultdict(list)

def add_results(ind, results):
    results['gen'].append(ind.gen) 
    results['ind'].append(ind.num)
    results['total_pf'].append(ind[0])
    results['poly'].append([ind[1],ind[2],ind[3],ind[4]])
    results['keff'].append(ind.keff)
    return results 

In [ ]:
pop_size = 20
pop = toolbox.population(n=pop_size)
# no. of generations, 
ngen, cxpb, mutpb = 10, 0.5, 0.5

# initialize the first population of individuals' fitness values 
ind_count = 0 
for ind in pop: 
    ind.gen = 0
    ind.num = ind_count
    ind_count += 1 
fitnesses = toolbox.map(toolbox.evaluate, pop)
dz = 10
dz_vals = np.arange(1,dz+1)
for ind, fitness in zip(pop, fitnesses):
    ind.fitness.values = (fitness[1],)
    ind.keff = fitness[0]
    results = add_results(ind,results)
    print(results)
    
for g in range(ngen):
    print('GENERATION', g)
    new_pop = []
    for ind in pop:
        #fitnesses = ind.fitness.values
        if ind.keff >= 1:
            new_pop.append(ind)
    # return half pop with lowest total PF 
    new_pop = toolbox.select(new_pop, k = pop_size)
    # extend new_pop back to 100 length 
    new_pop = list(islice(cycle(new_pop),pop_size))
    # mate some of them 
    for child1, child2 in zip(new_pop[::2], new_pop[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values, child2.fitness.values
    for mutant in new_pop:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values 
    ind_count = 0 
    for ind in new_pop: 
        ind.gen = g+1
        ind.num = ind_count
        ind_count += 1 
    fitnesses = toolbox.map(toolbox.evaluate, new_pop)
    dz = 10
    dz_vals = np.arange(1,dz+1)
    for ind, fitness in zip(new_pop, fitnesses):
        ind.fitness.values = (fitness[1],)
        ind.keff = fitness[0]
        results = add_results(ind,results)
        print(results)
    pop = new_pop.copy()
    pf_list = [] 
    for ind in pop: 
        pf_list.append(round(ind[0],4))
    print(pf_list)

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [ ]:
df = pd.DataFrame(data=results)

In [ ]:
df

In [ ]:
df.plot(x='gen', y='total_pf',kind='scatter')